# Offline learning for closed-loop setups

1. Overview
2. Purpose
3. Preparation
4. Implementation
    - Ridge regression
    - Chaotic time-series tasks
    - Offline training switching
5. Demonstration

## 1. Overview

Here, let's learn an **offline learning method** to train **linear closed-loops** in a reservoir computing (RC) setup. 
H. Jaeger *et al.*, (2004), one of the originators of RC, introduced closed-loop training in [his famous RC paper](https://www.science.org/doi/10.1126/science.1091277), demonstrating the outstanding performance of the echo state network (ESN) in predicting **chaotic time series data**.
The training process is almost the same as the open-loop one, except that you need to switch the loop connection according to the training phase.

In this notebook, we will master the closed-loop prediction with a simple discrete ESN (DESN) setup by replicating their results and understanding the way to apply the technique to control your own system.

## 2. Purpose
Implementing an offline training setup for linear loop and replicating the paper's results predicting chaotic time series data.

## 3. Preparation

In [ ]:
import sys
import math
import numpy as np

In [ ]:
if "google.colab" in sys.modules:
    from google.colab import drive  # Connecting to google drive
    drive.mount("/content/gdrive")
    %cd /content/gdrive/My Drive/sandbox/rc_tutorial/closed_loop_control/
    # NOTE: change it to your own path
    # e.g., %cd /content/gdrive/My Drive/[PATH_TO_NOTEBOOK]
    from utils.magic import colab_script  # save and load cells
    exec(colab_script)
else:
    sys.path.append(".")
    from utils.magic import run_and_save, load_and_run  # save and load cells

from utils.dde import ddeint # emulating delayed differential equation
from utils.tqdm import tqdm  # progress bar
from utils.reservoir import Module, ESN  # class for echo state network
from views.offline_learning import *  # visualization wrapper
from tests.offline_learning import *  # check function

## 4. Implementation
### 4.1 Ridge regression
First, let's implement a ridge regression for training closed-loop embedding desired attractors.

Suppose we have the following $N$-dimensional DESN setup with a linear readout $W^{\text{out}} \in \mathbb{R}^{D \times N}$ outputting $D$-dimensional vector and feeding it back to the system by a linear feedback loop $W^\text{feed} \in \mathbb{R}^{N \times D}$;
$$
\def\bm#1{{\boldsymbol #1}}
\begin{align*}
\\
\bm{x}(t+1) &= \tanh\left(g W^\text{net} \bm{x}(t) + W^\text{feed}\bm{y}(t)\right), \\ 
\bm{y}(t) &= W^{\text{out}} \bm{x}(t).
\\
\end{align*}
$$
Here, the objective of the closed-loop tasks is to approximate the desired function $\bm{d}(t)$ on the output by optimizing $W^{\text{out}}$; that is, the cost function $\mathcal{L}_{W^{\text{out}}}$ can be defined in the following mean square error;
$$
\begin{align*}
\\
\mathcal{L}_{W^{\text{out}}} &= \frac{1}{T} \sum_{t=1}^{T} \sum_{i=1}^{D} \left(\bm{y}_i(t) - \bm{d}_i(t)\right)^2 \\
&= \frac{1}{T} \cdot \text{RSS}.
\\
\end{align*}
$$
The optimal solution for this cost function can be obtained in a one-shot manner by the following simple linear regression method;
$$
\begin{align*}
\\
W^\text{out} &= \arg \min_{W^{\text{out}}} \mathcal{L}_{W^{\text{out}}} \\
&= \arg \min_{W^{\text{out}}} \|X {W^\text{out}}^T - Y\|^2\\
&= \left((X^T X)^{-1} (X^T D)\right)^T,
\\
\end{align*}
$$
where $X=[\bm{x}^T(1); \bm{x}^T(2); \cdots \bm{x}^T(T)]\in \mathbb{R}^{T\times N}$ and $D=[\bm{d}^T(1); \bm{d}^T(2); \cdots \bm{d}^T(T)] \in \mathbb{R}^{T\times D}$ are sampled datasets.
To avoid overfitting and causing unstable behavior owing to a large norm of $W^\text{out}$, a regularization parameter $\alpha (>0) \in \mathbb{R}$ is introduced to incorporate L2 norm in the cost function of **Ridge regression** as shown below;
$$
\begin{align*}
\\
\mathcal{L}_{W^{\text{out}}}^\text{Ridge} = \text{RSS} + \alpha \|W^\text{out}\|^2,
\\
\end{align*}
$$
which also has the following optimal solution given in a one-shot manner;
$$
\begin{align*}
W^\text{out} &= \arg \min_{W^{\text{out}}} \mathcal{L}_{W^{\text{out}}}^\text{Ridge} \\
&= \left((X^T X + \alpha I)^{-1} (X^T D)\right)^T,
\\
\end{align*}
$$
where $I \in \mathbb{R}^{N\times N}$ is an indentity matrix.
In the offline training method, we collect the dataset by constructing an open-loop setup replacing the system output $\bm{y}$ with the target $\bm{d}$ (a.k.a. **teacher forcing**).

First, let's fill in the blanks in `RidgeReadout.train`.

Specification of `RidgeReadout.train`
- Arguments
    - `X` (`np.ndarray`): Sampled reservoir states ($\in \mathbb{R}^{T\times N}$).
    - `Y` (`np.ndarray`): Target trajectories ($\in \mathbb{R}^{T\times D}$).
- Return
    - `np.ndarray`: Optimimal weight $W^\text{out}$ ($\in \mathbb{R}^{D \times N}$) for $\mathcal{L}^\text{Ridge}$.
- Operation
    - Update `RidgeReadout.w` with the obtained weight.

<details>
<summary>
Hints
</summary>

- Use [`numpy.linalg.lstsq`](https://numpy.org/doc/stable/reference/generated/numpy.linalg.lstsq.html) or [`scipy.linalg.lstsq`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.lstsq.html).

In [ ]:
%%run_and_save ./answer/linear.py

class Linear(Module):
    def __init__(self, input_dim, output_dim, bound=None, **kwargs):
        '''
        Linear model

        Args:
            input_dim (int): input node dim.
            output_dim (int): output node dim.
        '''
        super(Linear, self).__init__(**kwargs)
        self.input_dim = input_dim
        self.output_dim = output_dim
        if bound is None:
            bound = math.sqrt(1 / output_dim)
        self.w = self.rnd.uniform(
            -bound, bound, (output_dim, input_dim)).astype(self.dtype)

    def __call__(self, x):
        return self.predict(x)

    def predict(self, x):
        x = np.asarray(x)
        if x.ndim == 0:
            x = x.reshape(-1)
        return np.matmul(x, self.w.swapaxes(-1, -2))

    
class RidgeReadout(Linear):
    def train(self, X, Y, alpha=0):
        assert X.shape[0] == Y.shape[0]
        A = ... # TODO
        B = ... # TODO 
        if alpha > 0:
            dim = X.shape[1]
            A += ... # TODO
        sol, _residuals, _rank, _s = ... # TODO use lstsq
        self.w[:] = ... # TODO transpose is required when lstsq is used
        return self.w

You will see `OK!` if your code is correct.

In [ ]:
test_ridge_regression(RidgeReadout)

Next, let's design the emulation code for offline learning described above.
Fill in the blank to realize the switching.

Specification of `emulate`:
- Arguments:
    - `ts` (`list` or `np.ndarray`): List of sampled time (*e. g.*, [0, 1, 2, ..., T-1]).
    - `net` (`ESN`): An instance for DESN ($\tanh, W^\text{net}$).
    - `w_feed` (`Linear` or `RidgeReadout`): An instance of $W^\text{feed}$.
    - `w_out` (`Linear` or `RidgeReadout`): An instance of $W^\text{out}$.
    - `ds` (`list` or `np.ndarray`): Desired time series data $\bm{d}(t)$.
    - `open_range`: Time range of open-loop (*i.e.* training phase, $[t_0, t_1)$).

<details>
<summary>
Hints
</summary>

- When `ds` is `None` and $t \in [t_0, t_1)$, the script will feed the system output $\bm{y}$ back into the system.
- Otherwise, it will use `ds[idx]`.

In [ ]:
def emulate(ts, net, w_feed, w_out, ds=None, open_range=[0, 0], label=None):
    T = len(ts)
    record = {}
    record["t"] = np.zeros(T)
    record["x"] = np.zeros((T, *net.x.shape))
    record["y"] = np.zeros((T, *w_out(net.x).shape))
    record["open_range"] = open_range
    pbar = tqdm(ts)
    for idx, t in enumerate(pbar):
        if label is not None:
            pbar.set_description(label)
        if ds is not None and idx < len(ds) and open_range[0] <= t < open_range[1]:
            y = ... # TODO training phase (i.e., teacher forcing by ds)
        else:
            y = ... # TODO evaluation phase (i.e., autonomous mode by w_out)
        u = w_feed(y)
        record["t"][idx] = t
        record["x"][idx] = net.x
        record["y"][idx] = y
        net.step(u)
    return record

Now, it's time to check if the script properly works.
Let's build a closed-loop setup and visualize the dynamics. 

In [ ]:
net_dim = 1000
out_dim = 1

bias = np.random.uniform(-0.2, 0.2, net_dim)
net = ESN(net_dim, g=0.8, p=0.01, bias=bias)
w_feed = Linear(out_dim, net_dim)
w_out = RidgeReadout(net_dim, out_dim)

You will see a transition around the switching period

In [ ]:
time_steps = 200
open_range = (0, 100)

ds = np.random.uniform(-1, 1, time_steps)
net.x = np.random.randn(net_dim)
record = emulate(range(200), net, w_feed, w_out, ds, open_range=open_range)
fig = show_record(record)

### 4.2 Chaotic time-series tasks
#### Runge-Kutta methods

Next, let's implement [**Runge-Kutta methods**](https://en.wikipedia.org/wiki/Runge%E2%80%93Kutta_methods) (a.k.a. RK4) to obtain chaotic time series with regard to a map $f$ (supporting both time-variant and invariant) in specified ordinary differential equations.

$$
\begin{align*}\\
\dot{\bm{x}}(t) &= f(\bm{x}(t), t) \\
\bm{x}(t+\Delta t) &\approx \bm{x}(t) + \frac{1}{6} \left(\bm{k}_1 + 2\bm{k}_2 + 2\bm{k}_3 + \bm{k}_4\right),
\\\end{align*}
$$
where $\bm{k}_1, \bm{k}_2, \bm{k}_3, \bm{k}_4$ are 
$$
\begin{align*}\\
\bm{k}_1 &= \Delta t \cdot f(\bm{x}(t), t) \\
\bm{k}_2 &= \Delta t \cdot f\left(\bm{x}(t) + \frac{\bm{k}_1}{2}, t + \frac{\Delta t}{2} \right) \\
\bm{k}_3 &= \Delta t \cdot f\left(\bm{x}(t) + \frac{\bm{k}_2}{2}, t + \frac{\Delta t}{2} \right) \\
\bm{k}_4 &= \Delta t \cdot f(\bm{x}(t) + \bm{k}_3, t + \Delta t). \\
\\\end{align*}
$$
Specification of `emulate`:
- Arguments:
    - `func` (`function`): A map of the system.
    - `x0` (`np.ndarray` or `list`): An initial state.
    - `ts` (`np.ndarray` or `list`): List of time steps (*i.e.,* $[t_0, t_1, \cdots, t_{T-1}]$)
- Return:
    - `np.ndarray`: Emulated time series data ($\in \mathbb{R}^{T \times \text{dim}(x_0)}$)


<details>
<summary>
Hints
</summary>

- `x` is current state.
- `func` takes two arguments.

In [ ]:
%%run_and_save ./answer/odeint.py

def odeint(func, x0, ts, **params):
    x = np.array(x0)
    ys = np.zeros((len(ts), *x.shape))
    ys[0] = x
    for idx, dt in enumerate(tqdm(np.diff(ts))):
        k1 = dt * func(x, ts[idx], **params)
        k2 = dt * func(x + 0.5 * k1, ts[idx] + 0.5 * dt, **params)
        k3 = dt * func(x + 0.5 * k2, ts[idx] + 0.5 * dt, **params)
        k4 = dt * func(x + k3, ts[idx] + dt, **params)
        x += (k1 + 2 * k2 + 2 * k3 + k4) / 6
        ys[idx + 1] = x
    return ys


In [ ]:
test_runge_kutta(odeint)

#### Lorenz system
We also prepare chaotic time series data from [Lorenz system](https://en.wikipedia.org/wiki/Lorenz_system).
$$
\begin{align*}\\
\frac{dx_1}{dt} &= a(x_2 - x_1),\\
\frac{dx_2}{dt} &= x_1 (b - x_3) - x_2,\\
\frac{dx_3}{dt} &= x_1 x_2 - c x_3,\\
\\\end{align*}
$$

Please fill the following blank in `lorenz_eqn`.

Specification of `lorenz_eqn`:
- Arguments:
    - `x` (`np.ndarray`): State (`x.shape`: `(3,)`).
    - `_t` (`float`): Time (will not be used since Lorenz system is time-invariant *i.e.*, autonomous).
- Return:
    - `np.ndarray`: $\dot{x}(t)$.

<details>
<summary>
Hints
</summary>

- the argument `x` is a three dimensional vector ($[x_1, x_2, x_3]$).

In [ ]:
%%run_and_save ./answer/lorenz_system.py

def lorenz_eqn(x, _t, a=10.0, b=28.0, c=8.0 / 3.0):
    x_dot = np.zeros_like(x)
    x_dot[0] = ... # TODO
    x_dot[1] = ... # TODO
    x_dot[2] = ... # TODO
    return x_dot

In [ ]:
test_lorenz(lorenz_eqn)

Let's check the famous double wing structure of the Lorenz system.

In [ ]:
ts = np.linspace(0, 200, 20001)
ys_lz = odeint(lorenz_eqn, [1.0, 1.0, 1.0], ts)

In [ ]:
fig = show_delayed_coord(ys_lz[:, 0], ys_lz[:, 1], ys_lz[:, 2], tau=10)

In [ ]:
fig = show_3d_coord(ys_lz)

In [ ]:
fig.show()

#### Mackey-Glass system
Next, let's prepare a chaotic time series data from [Mackey-Glass (MG) system](http://www.scholarpedia.org/article/Mackey-Glass_equation), which is governed by the following differential equation;
$$
\begin{align*}\\
\dot{x}(t) = \frac{0.2 \cdot x(t-\tau)}{1 + x(t-\tau)^{10}} - 0.1 \cdot x(t),
\\\end{align*}
$$
where $\tau$ is a delay parameter.
Generating chaotic behavior, the MG system with $\tau=17$ was used for the benchmark in the paper.

Specification of `mackey_glass_eqn`:
- Arguments:
    - `x` (`function`): $x(t)$.
    - `t` (`float`): Time.
- Return:
    - `np.ndarray`: $\dot{x}(t)$.


<details>
<summary>
Hints
</summary>

- Note that it is *not* `np.ndarray` or `list`. Please check `dde.py` for the detailed implementation
    - The argument `x` is function and you can use `x(t - tau)`.
- `values_before_zero` controls the value of $x(t)$ when $t$ is negative.


In [ ]:
%%run_and_save ./answer/mackey_glass.py

def mackey_glass_eqn(x, t, tau: float = 17):
    x_dot = ... # TODO
    return x_dot


def values_before_zero(t):
    return 0.5

In [ ]:
test_mackey_glass(mackey_glass_eqn)

Let's check the dynamics.
The following script calculates the time-series data of MG system
(it will take 20~30 seconds since it uses `ddeint` specialized for delayed differential equation).

In [ ]:
ts = np.linspace(0, 10000, 10001)
ys_mg = ddeint(mackey_glass_eqn, values_before_zero,
               ts, atol=1e-16, rtol=1e-16)

Let's visualize the attractor structure by constructing the delayed coordinates.




In [ ]:
fig = show_delayed_coord(ys_mg, tau=17)

In [ ]:
data = construct_delayed_coord(ys_mg[:5000], 17, 3)
fig = show_3d_coord(data)

In [ ]:
fig.show()

### 4.3 Offline training switching
Next, let's prepare an offline training setup with Ridge regression.

Please fill the following in `run_train_and_eval`.

<details>
<summary>
Hints
</summary>

- When `ds` is `None` and $t \in [t_0, t_1)$, this function will feed the system output $\bm{y}$ back into the system.
- Otherwise, it will use `ds[idx]`.

In [ ]:
def run_train_and_eval(
        net, w_feed, w_out, ds, T_washout, T_train, T_eval, alpha=1e-8):
    # training phase (open loop)
    record_t = emulate(
        range(T_washout + T_train), net, w_feed, w_out, ds,
        open_range=[0, T_washout + T_train], label="Train")
    x_offset = np.array(net.x)
    fig_t = show_record(record_t)
    fig_t[0].set_title("Training")

    X_train = record_t["x"][T_washout:T_washout + T_train]
    Y_train = record_t["y"][T_washout:T_washout + T_train]
    Y_eval = ds[T_washout + T_train:T_washout + T_train + T_eval]

    # TODO run ridge_regression and update the weight
    ...

    # evaluation phase (closed loop)
    net.x[:] = x_offset
    record_e = emulate(
        range(T_eval), net, w_feed, w_out, label="Eval")
    record_e["d"] = Y_eval

    fig_e = show_record(record_e)
    fig_e[0].set_title("Evaluation")
    return record_t, record_e

## 5. Demonstration
### 5.1 Non-chaotic case

Now, it's time to solve chaotic time series prediction with ESN.

First, let's prepare the same DESN setup as described in the paper (if the result is poor, please regenerate the setup by running the next cell again).

In [ ]:
seed = 1234
rnd = np.random.RandomState(seed)

net_dim = 1000
out_dim = 3

bias = rnd.uniform(-0.2, 0.2, net_dim)
net = ESN(net_dim, g=1.0, p=0.01, bias=bias, a=None, rnd=rnd)
w_feed = Linear(out_dim, net_dim, bound=1.0, rnd=rnd)
w_out = RidgeReadout(net_dim, out_dim, rnd=rnd)

First, let's try the Lorenz system prediction.

In [ ]:
T_washout, T_train, T_eval = 2000, 8000, 6000
ds = ys_lz * 0.01

net.x = rnd.randn(net_dim)
rec_lz_t, rec_lz_e = run_train_and_eval(
    net, w_feed, w_out, ds, T_washout, T_train, T_eval, alpha=1e-6)

In [ ]:
fig = show_3d_coord(output=rec_lz_e["y"], desired=rec_lz_e["d"])

In [ ]:
fig.show()

In [ ]:
fig = show_delayed_coord(rec_lz_e["y"][:, 0], rec_lz_e["d"][:, 0], tau=17, labels=["output", "desired"])

Next, how about the MG system prediction?
(Note that the following processes reused `net.w_net` and `net.bias`.)

In [ ]:
out_dim = 1

w_feed = Linear(out_dim, net_dim, bound=1.0, rnd=rnd)
w_out = RidgeReadout(net_dim, out_dim, rnd=rnd)

T_washout, T_train, T_eval = 2000, 6000, 2000
ds = ys_mg - 1

net.x = rnd.randn(net_dim)
rec_mg_t, rec_mg_e = run_train_and_eval(
    net, w_feed, w_out, ds, T_washout, T_train, T_eval, alpha=1e-8)


In [ ]:
data1 = construct_delayed_coord(rec_mg_e["y"], 17, 3)
data2 = construct_delayed_coord(rec_mg_e["d"], 17, 3)
fig = show_3d_coord(output=data1, desired=data2)

In [ ]:
fig.show()

In [ ]:
fig = show_delayed_coord(rec_mg_e["y"], rec_mg_e["d"], tau=17, labels=["output", "desired"])

### 5.2 Chaotic case

To compare with the FORCE learning results in the following notebook, let's check the performance of the offline learning method when training a closed loop in a chaotic setup.

In [ ]:
seed = 1234
rnd = np.random.RandomState(seed)

net_dim = 1000
out_dim = 3

bias = rnd.uniform(-0.2, 0.2, net_dim)
net = ESN(net_dim, g=1.5, p=0.01, bias=bias, a=None, rnd=rnd)  # NOTE sr is set to 1.5
w_feed = Linear(out_dim, net_dim, bound=1.0, rnd=rnd)
w_out = RidgeReadout(net_dim, out_dim, rnd=rnd)

T_washout, T_train, T_eval = 2000, 8000, 6000
ds = ys_lz * 0.01

net.x = rnd.randn(net_dim)
rec_lz_t, rec_lz_e = run_train_and_eval(
    net, w_feed, w_out, ds, T_washout, T_train, T_eval, alpha=1e-6)

fig = show_3d_coord(output=rec_lz_e["y"], desired=rec_lz_e["d"])

In [ ]:
fig.show()

## Advanced topics

- Read Jaeger's *Supporting Online Material* in the original paper
- Implement a "refined version" of the offline learning
  - They said the performance would be improved in some semi-supervised manner
- Improve the performance by grid search or some meta-learning method